# Abstraction layer to access storage

Only Minio storage is enabled at the moment, new storage providers could be added on a need to need basis i.e. azure, nfs, hdfs
The approach to storage should follow basic principles of buckets:
- Buckets are the main container in wich folders and files are uploaded (for nfs that would be the first level folders)
- Buckets should hold uac, this is not implemented at the moment
- From buckets can be created or listed folders and files

In [1]:
jr_path = str(os.environ["JUPYTER_ROOT"] + "/importer.py")
%run $jr_path

Appending finder


Import used contracts 

In [2]:
from Libs.Storage.Contracts import (folder_info, file_info)

Libs.Storage.Contracts
['/jupyter/Libs/Storage']
after split: Contracts
path: 
['/jupyter/Libs/Storage']
/jupyter/Libs/Storage/Contracts.ipynb
after split: Contracts
path: 
['/jupyter/Libs/Storage']
/jupyter/Libs/Storage/Contracts.ipynb
importing Jupyter notebook from /jupyter/Libs/Storage/Contracts.ipynb


This is the meta class, it validated that the implementations follow the proper shape. Type checking is done here.

In [3]:
class ClientMeta(type):
    """A Parser metaclass that will be used for parser class creation.
    """
    def __instancecheck__(cls, instance):
        return cls.__subclasscheck__(type(instance))

    def __subclasscheck__(cls, subclass):
        return (hasattr(subclass, 'authenticate') and 
                callable(subclass.authenticate) and
                hasattr(subclass, 'list_buckets') and 
                callable(subclass.list_buckets) and
                hasattr(subclass, 'create_bucket') and 
                callable(subclass.create_bucket) and
                hasattr(subclass, 'remove_bucket') and 
                callable(subclass.remove_bucket) and
                hasattr(subclass, 'list_folders') and 
                callable(subclass.list_folders) and
                hasattr(subclass, 'list_files') and 
                callable(subclass.list_files) and
                hasattr(subclass, 'get_content') and 
                callable(subclass.get_content) and
                hasattr(subclass, 'set_content') and 
                callable(subclass.set_content) and
                hasattr(subclass, 'remove_content') and 
                callable(subclass.remove_content))

The interface implements the meta class rules, is doesn't display methods but this is what should be inherited by implementations.

In [4]:
class StorageClientInterface(metaclass=ClientMeta):
    pass

The ClientType enum defines the types of providers that are supported. If there is a need for a new type, it should be added in the enum so that it can be managed by the libraries.

In [8]:
from enum import Enum
class ClientType(Enum):
    Default=0
    Minio=1

The libraries are imported to be used by the factory method. Providers should be validated against the interface in order to make sure they follow the interface model.

In [15]:
from Libs.Storage.MinioProvider import (minio_client)
if not issubclass(minio_client, StorageClientInterface):
    raise Exception("minio_client should implement StorageClientInterface")

To create a new storage client, call the factory method with the proper provider. If the wrong value is passed, an exception will be thrown. ex:
`cli = create_client(ClientType.Default)`

In [11]:
def raise_(ex):
    raise ex

def create_client(ctype: ClientType = ClientType.Default):
    switcher = {
        ClientType.Default: lambda : minio_client(),
        ClientType.Minio: lambda : minio_client()
    }
    
    factory = switcher.get(ctype, lambda: raise_(Exception("Wrong ClientType Selected")))
    return factory()


In [14]:
#cli = create_client(ClientType.Default)